

#Romanian sub-dialect identification 

####*Proiect realizat de Cozma Laura-Elena, grupa 244*
---

Proiectul constă în clasificarea știrilor din datele de test în două categorii, scrise în dialect moldovenesc sau românesc. Pentru aceasta, am folosit pentru procesarea textului modelul Tf-Idf, iar clasificatorul utilizat este ComplementNB. 

###Prezentarea algoritmului

Prima parte se bazează pe citirea știrilor. Aceasta este realizată cu funcția readLines, ce returnează un vector de stringuri, fiecare reprezentând o știre.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np

#read the train samples
file = open('/content/drive/My Drive/data/train_samples.txt')
train_samples = file.readlines()          #train_samples is now an array of tweets
print("Numar de stiri in train_samples: " , len(train_samples))
#read the test samples
file = open('/content/drive/My Drive/data/test_samples.txt')
test_samples = file.readlines()           #test_samples is now an array of tweets
print("Numar de stiri in test_samples: " , len(test_samples))
#read the validation samples
file = open('/content/drive/My Drive/data/validation_samples.txt')
validation_samples = file.readlines()     #validation_samples is now an array of tweets
print("Numar de stiri in validation_samples: " , len(validation_samples))

Numar de stiri in train_samples:  7757
Numar de stiri in test_samples:  2623
Numar de stiri in validation_samples:  2656


Pentru a extrage etichetele stirilor necesare formării documentului csv, am definit funcția extract_indexes_for_test. Aceasta va împarți fiecare știre în 2 elemente, după prima apariție a '\t' și va adăuga în vector primul element(adică id-ul știrii).


In [9]:
def extract_indexes_for_test(file):
   indexes = []
   for line in file:
       indexes.append(line.split('\t' , 1)[0])
   return indexes

indexes = extract_indexes_for_test(test_samples)
print("Primele 10 id-uri ale stirilor din test_samples sunt: " )
print(indexes[:10])

Primele 10 id-uri ale stirilor din test_samples sunt: 
['110499', '101319', '108883', '100925', '110852', '109538', '108874', '100082', '110470', '109725']


După ce am citit și datele din fișierele cu predicții, apelez funcția extract_predictions. Aceasta returnează un vector cu predicții, care ne va ajuta la apelul clasificatorului și la calcularea f1 score. Extragerea predicției este similară extragerii etichetei, doar că în acest caz se extrage al doilea element rezultat dupa split.

In [8]:
def extract_predictions(file):
    labels = []
    for line in file:
      label = line.split('\t' , 1)[1]
      #every number has in the end a '\n' and we want to remove it
      label = label.replace('\n' , '')
      label_number = int(label)
      labels.append(label_number) 
    return labels

# read the train labels
file = open('/content/drive/My Drive/data/train_labels.txt')
train_labels = file.readlines()          #train_labels is now an array of "id prediction"
#extract the predictions
train_labels = extract_predictions(train_labels)
print("Primele 10 predictii pe datele de train sunt:")
print(train_labels[:10])

# read the validation labels
file = open('/content/drive/My Drive/data/validation_labels.txt')                                        
validation_labels = file.readlines()     #validation_labels is now an array of "id prediction"
#extract the predictions
validation_labels = extract_predictions(validation_labels)
print("Primele 10 predictii pe datele de validare sunt:")
print(validation_labels[:10])

Primele 10 predictii pe datele de train sunt:
[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Primele 10 predictii pe datele de validare sunt:
[0, 0, 1, 1, 0, 1, 0, 1, 1, 1]



##**Text mining:**
Pentru reprezentarea știrilor am folosit modelul TF-IDF(Term Frequency - Inverse Document Frequency). Acesta constă în asocierea fiecărui cuvânt dintr-un document o pondere, reprezentând importanța cuvântului în text.

Formula de calcul a numărului asociat fiecărui termen este:

####Tf-Idf = Term Frequency * Inverse Document Frequency


**Term Frequency:**

Se calculează frecvența unui cuvânt într-un document, după care se împarte la numarul total de cuvinte din documentul respectiv.

**Inverse Document Frequency:**

Se obține împărțind numărul de total de documente la numărul de documente în care cuvântul respectiv este prezent. Doar că în cazul unui număr mare de documente, valoarea este prea mare, de aceea se calculează logaritm din aceasta. În cazul în care un cuvânt nu se găsește în documente nu putem împarți la 0, de aceea se adună 1 la numărul de documente în care apare cuvântul.



####**TfidfVectorizer**




#####**Parametrii funcției:**

1. **Preprocessing:**

În mod implicit, TfidfVectorizer în faza de procesare desparte cuvintele în funcție de semnele de punctuație. Cum limbajul este criptat, nu putem împărți textul astfel, deoarece unele litere pot fi reprezentate ca ;, : etc. De aceea am construit o funcție numită process_data. Singura modificare pe care aceasta o face asupra tweet-urilor este eliminarea id-ul de la începutul lor.


In [0]:
#preprocessor for TfidfVectorizer
def process_data(line):
    #remove the index at the beginning of the tweet
    string_without_index = re.sub(r'\d+', ' ', line) 
    return string_without_index


2. **Lowercase**

Am setat cu False pentru nu transforma literele mari în litere mici, deoarece limbajul este criptat și 'a' poate fi diferit de litera 'A'.

3. **Token pattern**

Primește forma regex "\S\S+", astfel ignorându-se succesiunile de două sau mai multe spații.

4. **Norm**

Am lăsat valoarea implicită, adică 'l2', întrucât am observat o performanță mai bună decât în cazul 'l1'.

In [0]:
tfidf = TfidfVectorizer(preprocessor = process_data, lowercase = False, token_pattern = r"\S\S+")


În cazul parametrilor, mai puteam seta max_df(ignoră termenii care au frecvența în document mai mare decât acel prag), min_df(ignoră termenii care au frecvența în document mai mică decât acel prag) sau max_features. Însă performanța scădea atibuind valori acestor parametri.
Un exemplu ar fi urmatorul:

min_df | max_df | max_features | f1 score
--- | --- | --- | ---
- | - | - | 0.7274
0.1 | - | - | 0.4364
- | 0.6 | - | 0.7274





####**tfidf.fit_transform**

Este apelată având ca parametru train_samples, care este o lista de stringuri, fiecare string reprezentând un tweet. *fit_transform* împarte fiecare string în cuvinte, îl procesează conform preprocessor-ului și elimină cuvintele care au forma token pattern-ului. Pentru fiecare string face un vector de trăsături tf-idf. În final se returnează o matrice de trasături tf-idf pe care o vom folosi la antrenarea clasificatorului ComplementNB.

####**tfidf.transform**
Construiește matricea de trăsături pe baza vocabularului generat în antrenare.


In [13]:
train_samples_transformed = tfidf.fit_transform(train_samples)

print("Vocabularul este:\n" , tfidf.vocabulary_)

# process test_data
test_samples_transformed = tfidf.transform(test_samples)           #now test_samples_transformed is a matrix of features
print("Dimensiunile lui test_samples_transformed: " ,test_samples_transformed.shape)

# process validation_samples
validation_samples_transformed= tfidf.transform(validation_samples) #now validation_samples_transformed is a matrix of features
print("Dimensiunile lui validation_samples_transformed: " ,validation_samples_transformed.shape)

Vocabularul este:
 {';%fE': 2069, 'mr#&': 26002, 'crmx': 15445, 'temjc@m': 29446, "%'wb:": 943, '}hHAm@@m': 31028, 'ykm=aa': 30256, 'Eje@': 9179, 'Ejh=': 9200, 'EcrZk': 7466, 's}lZ$': 29228, 'rhfh': 28326, '}h@kofe@mk': 30905, 'RgWE<': 11463, '>mfor@m': 2736, '@#@': 2996, 'm=hkaa': 23226, 'TFr>o*': 11568, 'h}Ah': 21285, 'EHfm}e@mHk': 6977, 'e#hj@': 15671, 'j&}k': 21751, 'gAmaH': 18140, 'mgmkafe': 24553, 'cmT:': 15357, 'k>.h': 22830, 'XH(q!': 11779, '}FW': 30610, '@*oDgB': 3133, '#Sx.W': 210, 'hZ': 20143, 'jh=': 22156, 'chrZ': 15246, '}k#h': 31892, 'svcNt': 29224, 'ejmc@m': 16871, 'gYAmZ': 18296, 'efke@m': 16083, 'g@@m': 17964, '>m&': 2572, '}%k': 30509, 'tr(:': 29559, ';wxq': 2114, 'Ere': 9642, 'E*}ga': 6108, 'hgZ': 20585, 'h$mhr@m': 19450, 'tkafe@m': 29521, 't@A': 29291, '%#sE': 938, '=hkaa@m': 2311, 'm*gH': 23196, 'E@he=@m': 6407, 'wk}hX': 29774, 'Ejhr=@m': 9230, 'h@mg:@': 19835, 'sAFW': 29190, 'K#xk}t': 11305, 'fH@ae': 17399, 'm&Xd': 23172, '>h&': 2515, '@#': 2968, 'l@Rd}a': 23087, 


##**Clasificatorul Complement Naive Bayes**

Este asemănător MultinomialNB, doar că se bazează pe apartenența cuvântului în clasa complementară. Se consideră că parametrii pentru ComplementNB dau rezultate mult mai stabile decât la MultinomialNB. Parametrul alpha are rolul de a "netezi" clasificarea. Valoarea 0 corespunde lipsei netezirii.

In [16]:
from sklearn.naive_bayes import ComplementNB

complement_nb = ComplementNB(alpha = 0.2)
complement_nb.fit(train_samples_transformed, train_labels) 

print("Predict the labels...")

predicted_labels_validation = complement_nb.predict(validation_samples_transformed)
predicted_labels_test = complement_nb.predict(test_samples_transformed)

print("Primele 10 etichete prezise la datele de validare:")
print(predicted_labels_validation)
print("Primele 10 etichete prezise la datele de test:")
print(predicted_labels_test)

Predict the labels...
Primele 10 etichete prezise la datele de validare:
[0 0 1 ... 1 0 0]
Primele 10 etichete prezise la datele de test:
[0 0 0 ... 1 1 0]



Alegerea valorii alpha a fost stabilită prin multiple încercări.
Pentru calculul f1 score am folosit:
```
#compute f1 score on validation_samples
from sklearn.metrics import f1_score
print('f1 score: ' , f1_score(np.asarray(validation_labels) , predicted_labels_validation))
```
Pentru calculul acurateții am folosit:
```
def compute_accuracy(labels , predicted_labels):
  accuracy = np.sum(predicted_labels == labels) / len(predicted_labels)
  return accuracy

accuracy = compute_accuracy(np.asarray(validation_labels) , predicted_labels_validation)
print("Accuracy: " , accuracy * 100)
```
 Diverse valori pentru parametrul alpha împreună cu f1 score-ul asociat și acuratețea pentru validation_samples sunt:

Alpha | f1 score | accuracy 
--- | --- | ---
0.03 | 0.7362716763005780 | 72.515%
0.04 | 0.7368043087971274 | 72.402%
0.07 | 0.7355253752680486 | 72.138%
0.1  | 0.7323539949838768 | 71.875%
0.2  | 0.7274678111587983 | 71.310%
0.3  | 0.7252510760401721 | 71.159%
0.35 | 0.7240014393666788 | 71.121%
0.5  | 0.7212996389891696 | 70.933%
0.7  | 0.7182280319535221 | 70.783%
1    | 0.7122986822840410 | 70.406%
1.5  | 0.7001488095238095 | 69.653%
3    | 0.6784073506891270 | 68.373%
10   | 0.6334431630971994 | 66.490%
20   | 0.5896551724137932 | 64.156%

Se observă că f1 score crește cu cât alpha e mai mic, până se ajunge la 0.04, unde atinge valoarea maximă. Am ales alpha = 0.2 pentru a avea f1 score cât mai mare, dar și pentru a elimina suspiciunea că pentru valorile prea apropiate de 0 se face overfitting.

###**Matricea de confuzie**

Pentru alpha = 0.2, pentru care am obținut f1-score = 0.72746, matricea de confuzie este următoarea:


In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(validation_labels , predicted_labels_validation)

array([[ 877,  424],
       [ 338, 1017]])

Pentru alpha = 0.35, pentru care f1-score = 0.724, matricea de confuzie ia următoarea formă:

[ [ 883  418]

  [ 349 1006] ]

Din nou se observă că pentru alpha = 0.2 avem un număr mai bun de predicții.


###**Comparație între ComplementNB și alți clasificatori**
####**1. Linear SVC**
Obiectivul lui LinearSVC este sa clasifice datele pe baza hiperplanului care împarte datele în cele două categorii cel mai bine. Pentru un C mare(la parametru) marginea hiperplanului va fi mai mică, iar pentru un C mic o margine mai mare. La folosirea SVM s-a observat o performanță destul de scăzută în comparație cu Naive Bayes.
```
from sklearn import svm

svm_model = svm.LinearSVC(C=1)
svm_model.fit(train_samples_transformed, train_labels)

predicted_labels_validation = svm_model.predict(validation_samples_transformed)
predicted_labels_test = svm_model.predict(test_samples_transformed)
```
C | LinearSVC f1-score |  LinearSVC accuracy  
 --- | --- | --- 
 0.05 | 0.708150152181264 | 67.50753%
 1 | 0.691053965954364 | 67.88403%
 2 | 0.682429974536194 | 67.13102%
 10 | 0.672999634636463 | 66.30271%

####**2. Retele neuronale**
```

def train_and_test_the_model(model, train_features, train_labels, test_features, test_labels):
    model.fit(train_features, train_labels)
    print('Acuratetea pe validare: ', model.score(test_features, test_labels))


mlp_classifier_model = MLPClassifier(hidden_layer_sizes=(20), activation='relu', solver='sgd', learning_rate_init=0.006, momentum=0.9, max_iter=200, alpha=0.003)#, early_stopping = True, tol=1e-17, n_iter_no_change=50
train_and_test_the_model(mlp_classifier_model, train_data_vect, train_labels, validation_samples_vect, validation_labels)
```
În urma executării codului s-a obținut f1 score = 0.7018 și o acuratețe de 68.9%. La orice altă combinație de parametri se obțineau valori similare, semnificativ mai mici decât în cazul ComplementNB.

####**3. MultinomialNB**

Deși sunt similari, următorul tabel ilustrează câteva diferențe:

Alpha | ComplementNB f1-score | MultinomialNB f1-score |  ComplementNB accuracy  |  MultinomialNB accuracy
--- | --- | --- | --- | ---
0.2 |   0.7274678111587983 |0.7326033203814907 | 71.310%  | 71.498%
0.35 | 0.7240014393666788 |0.7294703163881977 | 71.121% | 71.347%

Chiar dacă valorile sunt mai mari în cazul MultinomialNB, pe 20% din datele de testare pentru alpha = 0.2 se obținea f1 score = 0.73936 cu MultinomialNB și f1 score = 0.74343 cu ComplementNB, ceea ce a determinat alegerea clasificatorului ComplementNB. 




##**Bibliografie**
1. https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089
2. https://appliedmachinelearning.blog/2017/02/12/sentiment-analysis-using-tf-idf-weighting-pythonscikit-learn/?fbclid=IwAR2Bwk8eJS_Ts7HZBM5yRBjn2X9BCEMPeZr3M6pyz7c3mfilatQMMixWZ1g
3. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
4. https://blog.floydhub.com/naive-bayes-for-machine-learning/
5. https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html?fbclid=IwAR3K8hHLIWOxjsgUj5nqYczGtcILKsWhy_bvpCvwyq5rQbFj-AsuH7yX404
6. https://stats.stackexchange.com/questions/31066/what-is-the-influence-of-c-in-svms-with-linear-kernel